In [2]:
# NYC Yellow Taxi Trip Analysis (September 2020)

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_format, hour, dayofweek, avg, count, when


In [3]:
# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("NYC Yellow Taxi September 2020 Analysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

In [5]:
# Step 2: Load Dataset
df = spark.read.csv("yellow_tripdata_2020-09.csv", header=True, inferSchema=True)

In [6]:
# Step 3: Convert datetime columns to timestamp
df = df.withColumn("pickup_datetime", to_timestamp(col("tpep_pickup_datetime"))) \
       .withColumn("dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime")))


In [7]:
# Step 4: Data Cleaning
df_clean = df.dropna(subset=["pickup_datetime", "dropoff_datetime", "passenger_count", "trip_distance", "total_amount"]) \
             .filter((col("trip_distance") > 0) & (col("total_amount") > 0))

In [8]:
# Step 5: Feature Engineering
trips = df_clean.withColumn("hour", hour(col("pickup_datetime"))) \
                .withColumn("day_of_week", dayofweek(col("pickup_datetime")))


In [9]:
# Step 6: Analysis

# 1. Trips per Day
trips_per_day = trips.groupBy(date_format("pickup_datetime", "yyyy-MM-dd").alias("date")).count()

# 2. Busiest Pickup Hours
busiest_hours = trips.groupBy("hour").count().orderBy("hour")

# 3. Average Fare by Trip Distance Range
trips = trips.withColumn("distance_range", 
    when(col("trip_distance") <= 2, "0-2 km")
    .when(col("trip_distance") <= 5, "2-5 km")
    .when(col("trip_distance") <= 10, "5-10 km")
    .otherwise("10+ km"))

avg_fare_by_distance = trips.groupBy("distance_range").agg(avg("total_amount").alias("avg_fare"))

# 4. Passenger Count vs Average Trip Distance
passenger_vs_distance = trips.groupBy("passenger_count").agg(avg("trip_distance").alias("avg_distance"))



In [10]:
# Step 7: Show Results
print("✅ Trips per Day:")
trips_per_day.show(5)

print("\n✅ Busiest Pickup Hours:")
busiest_hours.show()

print("\n✅ Average Fare by Distance Range:")
avg_fare_by_distance.show()

print("\n✅ Passenger Count vs Avg Trip Distance:")
passenger_vs_distance.show()



✅ Trips per Day:
+----------+-----+
|      date|count|
+----------+-----+
|2020-09-12|   34|
|2020-09-11|   50|
|2020-09-20|   24|
|2020-09-06|   16|
|2020-09-14|   20|
+----------+-----+
only showing top 5 rows

✅ Busiest Pickup Hours:
+----+-----+
|hour|count|
+----+-----+
|   0|   13|
|   1|    9|
|   2|    5|
|   4|    1|
|   5|    6|
|   6|   19|
|   7|   41|
|   8|   51|
|   9|   52|
|  10|   39|
|  11|   53|
|  12|   39|
|  13|   67|
|  14|   65|
|  15|   66|
|  16|   61|
|  17|   62|
|  18|   61|
|  19|   66|
|  20|   50|
+----+-----+
only showing top 20 rows

✅ Average Fare by Distance Range:
+--------------+------------------+
|distance_range|          avg_fare|
+--------------+------------------+
|        10+ km| 57.86423076923077|
|        0-2 km| 11.58889655172421|
|       5-10 km| 30.38507936507935|
|        2-5 km|18.704233870967776|
+--------------+------------------+


✅ Passenger Count vs Avg Trip Distance:
+---------------+------------------+
|passenger_count|      a

In [11]:
# Step 8: Stop Spark
spark.stop()